## Optimization Final Project
## Major League Baseball Expansion
#### Hank Snowdon, Joon Park, Toby Chiu, Vincent Feng
#### Winter 2024

#### 1. Loading in the data

In [1]:
using JuMP
using GLPK
using DataFrames
using PyPlot
using CSV
using LinearAlgebra
using Plots

In [2]:
# Import
# Data pulled from dataDownload.ipynb
player_data2 = CSV.read("../Data/player_data.csv", DataFrames.DataFrame, delim = ",", header = true)
player_data = player_data2

Row,IDfg,Name,Age,G,AB,WAR,name_last,name_first,key_bbref,mlb_played_first,2023,shortName,2024,SimplePosition,S2,G_all,GS,2024Est,Selectable,Position_c,Position_1b,Position_2b,Position_3b,Position_ss,Position_lf,Position_cf,Position_rf,Position_Reliever,Position_Starter,IP,Arizona,Atlanta,Baltimore,Boston,Chi Cubs,Chi White Sox,Cincinnati,Cleveland,Colorado,Detroit,Houston,Kansas City,LA Angels,LA Dodgers,Miami,Milwaukee,Minnesota,NY Mets,NY Yankees,Oakland,Philadelphia,Pittsburgh,San Diego,San Francisco,Seattle,St. Louis,Tampa Bay,Texas,Toronto,Washington
,Int64,String31,Int64,Int64,Int64,Float64,String15,String15,String15,Float64,Float64,String15,String7?,String31?,Bool,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,11493,Manny Machado,28,150,578,7.5,machado,manny,machama01,2012.0,2.10909e7,San Diego,$17.09,3b,false,150,148,1.709e7,1,0,0,0,1,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,9777,Nolan Arenado,30,148,557,7.2,arenado,nolan,arenano01,2013.0,3.28221e7,St. Louis,$35.00,3b,false,148,147,3.5e7,1,0,0,0,1,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,5361,Freddie Freeman,31,159,612,7.1,freeman,freddie,freemfr01,2010.0,2.52907e7,LA Dodgers,$25.29,1b,false,159,159,2.529e7,1,0,1,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9218,Paul Goldschmidt,33,151,561,7.0,goldschmidt,paul,goldspa01,2011.0,2.53333e7,St. Louis,$25.33,1b,false,151,150,2.533e7,1,0,1,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,19556,Yordan Alvarez,24,135,470,6.7,álvarez,yordan,alvaryo01,2019.0,7.83333e6,Houston,$10.83,lf,false,135,133,1.083e7,1,0,0,0,0,0,1,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,5417,Jose Altuve,31,141,527,6.6,altuve,josé,altuvjo01,2011.0,2.92e7,Houston,$29.20,2b,false,141,137,2.92e7,1,0,0,1,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,13611,Mookie Betts,28,142,572,6.6,betts,mookie,bettsmo01,2014.0,2.11587e7,LA Dodgers,$24.77,"2b,rf",false,142,138,2.477e7,1,0,0,1,0,0,0,0,1,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,12916,Francisco Lindor,27,161,630,6.6,lindor,francisco,lindofr01,2015.0,3.24773e7,NY Mets,$32.48,ss,false,161,161,3.248e7,1,0,0,0,0,1,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,11739,J.T. Realmuto,30,139,504,6.5,realmuto,j. t.,realmjt01,2014.0,2.3875e7,Philadelphia,$23.88,c,false,139,132,2.388e7,1,1,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


#### 2. Model Setup

### Decision Variables
- $x_{ij}$: Binary variable, 1 if player $i$ from team $j$ is protected, otherwise 0.
- $y_{i}$: Binary variable, 1 if player $i$ is selected by the expansion team, otherwise 0.


### Objective Functions
**Protection Model:**
$$ \max \sum_{i \in I} x_{ij} \cdot WAR_{i} $$

**Selection Model:**
$$ \max  \sum_{i \in I} y_{i} \cdot WAR_{i}  - \alpha \sum_{i \in I}  y_{i} \cdot 2024Salary_{i}   $$



**Constraints:**

- Salary Cap (Selection Models): $  \sum_{i \in I} y_{i} \cdot Salary_{i} \leq SalaryCap $

 
- Minimum number of SP and RP per selected team: pitchmin

### Constraints
**Salary Cap (Selection):**
$$ \sum_{i \in I} y_{i} \cdot Salary_{i} \leq SalaryCap $$

**Player Protection (Existing Teams):**
$$ \sum_{i \in I} x_{ij} \leq Protect, \forall j \in J $$

**Player Position (Selection):**
$$ PositionMin_{p} \leq \sum_{i \in P} x_{ij}  , \forall P \in Positions $$
$$ PitcherMin_{p} \leq \sum_{i \in P} x_{ij}  , \forall P \in Pitchers $$

**Player Drafting (Selection):**
$$ \sum_{i \in I} y_{i} = 35 $$

**Player Affiliation (Selection):**
$$ \sum_{i \in I_{j}} y_{i} \leq TeamLim, \forall j \in J $$

- $I$: Set of draft-eligible MLB players.
- $I_{j}$: Set of players available for drafting from MLB team $j$.
- $J$: Set of existing MLB teams.
- $Positions$: Set of field position pools.
- $Pitchers$: Set of pitcher position pools: SP and RP. 
- $WAR_{i}$: Wins Above Replacement value for player $i$.
- $Salary_{i}$: Annual salary of player $i$.
- $SalaryCap$: Total salary cap for the expansion team.
- $PositionMin_{p}$: Minimum number of players to be held at position $p$.
- $Protect$: Number of eligble players teams can protect from selection
- $TeamLim$: Maxmium number of players per team
- $PitcherMin$: Minimum number of players to be held at SP and RP.


#### 3. Single Round Draft

In [3]:
function simulation_basic(player_data, protect, teamlim, posmin, pitchmin, alpha )    
    
    ## PROTECTION FUNCTION
    # Each team chooses a list of players that they want to protect
     
    # Set teams
    teams_list = unique(player_data.shortName) 

    # Simple optimization problem for protection
    function protection_model(war,data)
        model = JuMP.Model(GLPK.Optimizer)

        @variable(model, x[1:length(war)], Bin)
        (((x, label),) -> set_name(x,label)).(zip(x,String.(data[!,9])))
    
        @constraint(model, sum(x[1:length(war)]) == protect)

        @objective(model, Max, war'x[1:length(war)])

        optimize!(model)
        return model, x
    end

    protected_players_dict = Dict{String, DataFrame}() 
    unprotected_players_df = DataFrame()

    function apply_protection(data)
        # Iterate over each team in teams_list
        for team in teams_list
            # Filter protectable players for the current team
            protectable_team_players = filter(row -> row.Selectable == 1 && row.shortName == team, data)
            WAR = protectable_team_players[!,"WAR"]

            # Apply protection model to select top x players by WAR
            model, x = protection_model(WAR, protectable_team_players)
            optimize!(model)
            
            # Check if optimization was successful
            if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
                protected_players_indices = findall(value.(JuMP.value.(x)) .== 1) 
                protected_players = protectable_team_players[protected_players_indices, :]
        
                # Filter players with Selectable == 0 for the current team
                non_selectable_team_players = filter(row -> row.Selectable == 0 && row.shortName == team, data)
                
                # Combine protected and non-protectable players into a single DataFrame
                team_players_df = vcat(protected_players, non_selectable_team_players)

                # Store the DataFrame for the current team in the dictionary
                protected_players_dict[team] = team_players_df

                # Filter out unprotected players for the current team
                unprotected_players_indices = setdiff(1:size(protectable_team_players, 1), protected_players_indices)
                unprotected_players = protectable_team_players[unprotected_players_indices, :]
                
                # Append unprotected players to the overall unprotected players DataFrame
                append!(unprotected_players_df, unprotected_players)
                
            else
                # Team has fewer than x players to protect 
                #println("No players to protect for $team")
                protected_players = protectable_team_players  
                non_selectable_team_players = filter(row -> row.Selectable == 0 && row.shortName == team, data) 
                team_players_df = vcat(protected_players, non_selectable_team_players)
            end 
        end
    end
    
    apply_protection(player_data)

    ## SELECTION FUNCTION
    # Setup optimization problem for selection
    function selection_model(dataframe)

        salary = dataframe[:,"2023"]
        WAR_remaining = dataframe[:,"WAR"]
        future_salary = dataframe[:,"2024Est"] 

        model = JuMP.Model(GLPK.Optimizer)

        @variable(model, x[1:length(WAR_remaining)], Bin) # Whether the player was chosen or not

        (((x, label),) -> set_name(x,label)).(zip(x,String.(dataframe[!,9]))) # Label each player with their id 

        @constraint(model, sum(x[1:length(WAR_remaining)]) .== 35) # Player Drafting = 35 total players
        @constraint(model, sum(salary .* x[1:length(salary)]) .<= 223000000) # Making sure it doesn't go over the salary cap of 223 Million

        # Making sure there is at max x players taken from each team 
        for col in 31:60
            @constraint(model, sum(dataframe[!, col] .* x) .<= teamlim)
        end  

        # Making sure there is at least x players per position
        for col in 20:27
            @constraint(model, sum(dataframe[!, col] .* x) .>= posmin)
        end     

        # Making sure there are enough pitchers
        for col in 28:29
            @constraint(model, sum(dataframe[!, col] .* x) .>= pitchmin)
        end     
    
        # Objective function: max WAR and financial flexibility
        @objective(model, Max, dot(WAR_remaining, x[1:length(WAR_remaining)])   - alpha  *   dot(future_salary, x[1:length(WAR_remaining)]))
        
        optimize!(model)  
        return model, x
    end
    
    selection_model2, x = selection_model(unprotected_players_df)
    term_status = JuMP.termination_status(selection_model2)
    selected_players_indices = findall(value.(JuMP.value.(x)) .== 1) 
    selected_players = unprotected_players_df[selected_players_indices, :] 
    total_war = sum(selected_players.WAR)
    caphit24 = sum(selected_players[!, "2024Est"])
    clean_selected_players = select(selected_players, [:Name, :WAR,  :shortName, :SimplePosition])

    return selected_players, clean_selected_players, term_status , protected_players_dict,  total_war, caphit24
end

simulation_basic (generic function with 1 method)

In [4]:
# Optimization model run for most realistic parameters
mod1 = simulation_basic(player_data, 12, 3, 3, 8, 0)   
print(mod1[2], "\n\n", mod1[5], "\n\n", mod1[6])
CSV.write("../Results/mainselection.csv", mod1[2])

35×4 DataFrame
 Row │ Name                  WAR      shortName      SimplePosition    
     │ String31              Float64  String15       String31?         
─────┼─────────────────────────────────────────────────────────────────
   1 │ Nick Martinez             0.4  San Diego      Starter
   2 │ Chris Stratton            0.7  St. Louis      Reliever
   3 │ David Peralta             1.7  LA Dodgers     lf
   4 │ Austin Barnes             1.4  LA Dodgers     c
   5 │ Miguel Rojas              1.0  LA Dodgers     1b,ss
   6 │ Daniel Vogelbach          1.4  NY Mets        missing           
   7 │ Luis Guillorme            1.3  NY Mets        2b,3b,ss
   8 │ Omar Narvaez              1.1  NY Mets        c
   9 │ Seranthony Dominguez      1.0  Philadelphia   Reliever
  10 │ Andrew Bellatti           1.0  Philadelphia   Reliever
  11 │ Craig Kimbrel             0.9  Philadelphia   Reliever
  12 │ Yan Gomes                 0.6  Chi Cubs       c
  13 │ Adbert Alzolay            0.5  Chi Cubs

"mainselection.csv"

In [16]:
# Test financial flexibility

function run_simulation_for_alpha(alpha) 
    selected_players, clean_selected_players, term_status, protected_players_dict, total_war, caphit24 = simulation_basic(player_data, 12, 3, 3, 8, alpha) 
    result_df = DataFrame(alpha = alpha, total_war = total_war, caphit24 = caphit24)
    return result_df
end
 
alpha_values = [0, 0.000000005, 0.00000001, 0.000000015, 0.000000025, 0.000000035, 0.00000005, 0.000000075, 0.0000001, 0.00000015, 0.0000002, 0.0000003, 0.0000004, 0.0000005, 0.0000006, 0.0000007, 0.0000008, 0.0000009, 0.000001,  0.0000011,  0.0000012,  0.0000013,  0.0000014,  0.0000015,  0.0000016,  0.0000017,       0.000002, 0.001]
results_df = DataFrame(alpha = Float64[], total_war = Float64[], caphit24 = Float64[])
for alpha in alpha_values
    results_df = vcat(results_df, run_simulation_for_alpha(alpha))
end
println(results_df)

# Create a line chart
Plots.plot(  results_df.caphit24/1000000,  results_df.total_war, ylabel="Total WAR", xlabel="'24 Salary Obligations (MM)", label="", title="Total WAR vs Future (2024) Salary Obligations (MM)", linewidth=5, size=(900, 800))
annotation_text = "12 players protected,\nMax 3 players per team,\nMin 3 per position, 8SP, 8RP "
annotate!(107,27.5, Plots.text(annotation_text, :center, 16))
scatter!( results_df.caphit24/1000000,  results_df.total_war, label="", color=:blue)  
Plots.savefig("../Results/caphit.png" )

28×3 DataFrame
 Row │ alpha    total_war  caphit24  
     │ Float64  Float64    Float64   
─────┼───────────────────────────────
   1 │  0.0          31.2  1.37589e8
   2 │  5.0e-9       31.2  1.37589e8
   3 │  1.0e-8       31.2  1.37589e8
   4 │  1.5e-8       30.7  9.5939e7
   5 │  2.5e-8       30.5  8.6881e7
   6 │  3.5e-8       29.9  6.8126e7
   7 │  5.0e-8       29.0  4.7721e7
   8 │  7.5e-8       28.3  3.575e7
   9 │  1.0e-7       28.3  3.575e7
  10 │  1.5e-7       27.0  2.6239e7
  11 │  2.0e-7       25.1  1.529e7
  12 │  3.0e-7       23.9  1.0262e7
  13 │  4.0e-7       23.6  9.262e6
  14 │  5.0e-7       21.1  3.525e6
  15 │  6.0e-7       21.1  3.525e6
  16 │  7.0e-7       21.1  3.525e6
  17 │  8.0e-7       21.1  3.525e6
  18 │  9.0e-7       21.1  3.525e6
  19 │  1.0e-6       20.4  2.812e6
  20 │  1.1e-6       20.4  2.812e6
  21 │  1.2e-6       20.4  2.812e6
  22 │  1.3e-6       20.4  2.812e6
  23 │  1.4e-6       20.4  2.812e6
  24 │  1.5e-6       20.4  2.812e6
  25 │  1.6e-6     

"/Users/hank/Documents/UChicago/Optimization/OptimizationProject/Code/caphit.png"

In [38]:
# Test team limits 

function run_simulation_for_teamlimit(teamlim) 
    selected_players, clean_selected_players, term_status, protected_players_dict, total_war, caphit24 = simulation_basic(player_data, 12, teamlim, 3, 8, 0) 
    result_df = DataFrame(teamlim = teamlim, total_war = total_war )
    return result_df
end
 
teamlim_values  = [2,3,4,5,6,7,8,9,10]
results_df = DataFrame(teamlim = Float64[], total_war = Float64[] )
for teamlim in teamlim_values
    results_df = vcat(results_df, run_simulation_for_teamlimit(teamlim))
end
println(results_df)

# Create a line chart
Plots.plot(  results_df.teamlim,  results_df.total_war, ylabel="Total WAR", xlabel="Max Players Taken per Team", label="", title="Total WAR vs Max Players Taken per Team", linewidth=2)
annotation_text = "12 players protected,\nMin 3 per position, 8SP, 8RP "
annotate!(8.5,32, Plots.text(annotation_text, :center, 10))
scatter!( results_df.teamlim,  results_df.total_war, label="", color=:blue)
Plots.savefig("../Results/teamlim.png")

9×2 DataFrame
 Row │ teamlim  total_war 
     │ Float64  Float64   
─────┼────────────────────
   1 │     2.0       27.9
   2 │     3.0       31.2
   3 │     4.0       32.3
   4 │     5.0       33.1
   5 │     6.0       33.2
   6 │     7.0       33.2
   7 │     8.0       33.2
   8 │     9.0       33.2
   9 │    10.0       33.2


"/Users/hank/Documents/UChicago/Optimization/OptimizationProject/Code/teamlim.png"

In [41]:
# Test position mins

function run_simulation_for_posmin(posmin) 
    selected_players, clean_selected_players, term_status, protected_players_dict, total_war, caphit24 = simulation_basic(player_data, 12, 3, posmin, 8, 0) 
    result_df = DataFrame(posmin = posmin, total_war = total_war )
    return result_df
end
 
posmin_values  = [2,3,4,5 ]
results_df = DataFrame(posmin = Float64[], total_war = Float64[] )
for posmin in posmin_values
    results_df = vcat(results_df, run_simulation_for_posmin(posmin))
end
println(results_df)

# Create a line chart
Plots.plot(  results_df.posmin,  results_df.total_war, ylabel="Total WAR", xlabel="Min Players per Position", label="", title="Total WAR vs Min Players per Position", linewidth=2)
annotation_text = "12 players protected,\nMax 3 players per team,\nMin 8 SP, 8 RP "
annotate!(4.5,31.25, Plots.text(annotation_text, :center, 10))
scatter!( results_df.posmin,  results_df.total_war, label="", color=:blue)
Plots.savefig("../Results/posmin.png")

4×2 DataFrame
 Row │ posmin   total_war 
     │ Float64  Float64   
─────┼────────────────────
   1 │     2.0       31.7
   2 │     3.0       31.2
   3 │     4.0       30.0
   4 │     5.0       28.5


"/Users/hank/Documents/UChicago/Optimization/OptimizationProject/Code/posmin.png"

In [45]:
# Test pitcher mins 

function run_simulation_for_pitchmin(pitchmin) 
    selected_players, clean_selected_players, term_status, protected_players_dict, total_war, caphit24 = simulation_basic(player_data, 12, 3, 3, pitchmin, 0) 
    result_df = DataFrame(pitchmin = pitchmin, total_war = total_war )
    return result_df
end
 
pitchmin_values  = [2,3,4,5,6,7,8,9,10,11,12,13 ]
results_df = DataFrame(pitchmin = Float64[], total_war = Float64[] )
for pitchmin in pitchmin_values
    results_df = vcat(results_df, run_simulation_for_pitchmin(pitchmin))
end
println(results_df)

# Create a line chart
Plots.plot(  results_df.pitchmin,  results_df.total_war, ylabel="Total WAR", xlabel="Min Starters/Relievers", label="", title="Total WAR vs Min Starters/Relievers", linewidth=2)
annotation_text = "12 players protected,\nMax 3 players per team,\nMin 3 per position "
annotate!(11.5,31.35, Plots.text(annotation_text, :center, 10))
scatter!( results_df.pitchmin,  results_df.total_war, label="", color=:blue)
Plots.savefig("../Results/pitchmin.png")

12×2 DataFrame
 Row │ pitchmin  total_war 
     │ Float64   Float64   
─────┼─────────────────────
   1 │      2.0       32.0
   2 │      3.0       32.0
   3 │      4.0       32.0
   4 │      5.0       31.9
   5 │      6.0       31.7
   6 │      7.0       31.5
   7 │      8.0       31.2
   8 │      9.0       30.8
   9 │     10.0       30.4
  10 │     11.0       30.0
  11 │     12.0       29.0
  12 │     13.0       26.7


"/Users/hank/Documents/UChicago/Optimization/OptimizationProject/Code/pitchmin.png"

In [46]:
# Test num protected 

function run_simulation_for_protect(protect) 
    selected_players, clean_selected_players, term_status, protected_players_dict, total_war, caphit24 = simulation_basic(player_data, protect, 3, 3, 8, 0) 
    result_df = DataFrame(protect = protect, total_war = total_war )
    return result_df
end
 
protect_values  = [5,6,7,8,9,10,11,12,13,14,15,16,17 ]
results_df = DataFrame(protect = Float64[], total_war = Float64[] )
for protect in protect_values
    results_df = vcat(results_df, run_simulation_for_protect(protect))
end
println(results_df)

# Create a line chart
Plots.plot(  results_df.protect,  results_df.total_war, ylabel="Total WAR", xlabel="Players Protected Per Team", label="", title="Total WAR vs Players Protected Per Team", linewidth=2)
annotation_text = "Max 3 players per team,\nMin 3 per position, 8SP, 8RP"
annotate!(15,70, Plots.text(annotation_text, :center, 10))
scatter!( results_df.protect,  results_df.total_war, label="", color=:blue) 
Plots.savefig("../Results/protect.png")

13×2 DataFrame
 Row │ protect  total_war 
     │ Float64  Float64   
─────┼────────────────────
   1 │     5.0       85.4
   2 │     6.0       74.7
   3 │     7.0       66.2
   4 │     8.0       58.8
   5 │     9.0       51.0
   6 │    10.0       45.1
   7 │    11.0       37.7
   8 │    12.0       31.2
   9 │    13.0       25.5
  10 │    14.0       19.7
  11 │    15.0       15.0
  12 │    16.0        9.7
  13 │    17.0        3.6


"/Users/hank/Documents/UChicago/Optimization/OptimizationProject/Code/protect.png"

#### 4. Multi-Round Draft

In [52]:
player_data2 = CSV.read("../Data/player_data.csv", DataFrames.DataFrame, delim = ",", header = true)
player_data = player_data2

selected_IDs = Vector{Any}()
rounds_data = player_data 

selected_players_byround_dict = Dict{Int, DataFrame}() 


function simulation_rounds(player_data, protect_tuple, teamlim, posmin, pitchmin, alpha, rounds, picks_tuple )  
    
    total_salary = 0 
    
    teams_list = unique(player_data.shortName)

    function protection_model(war,data, protect)
        model = JuMP.Model(GLPK.Optimizer)

        @variable(model, x[1:length(war)], Bin)
        (((x, label),) -> set_name(x,label)).(zip(x,String.(data[!,9])))
    
        @constraint(model, sum(x[1:length(war)]) == protect)

        @objective(model, Max, war'x[1:length(war)])

        optimize!(model)
        return model, x
    end 

    ## SELECTION FUNCTION
    function selection_model(dataframe, picks)
        salary = dataframe[:,"2023"]
        WAR_remaining = dataframe[:,"WAR"]
        future_salary = dataframe[:,"2024Est"] 
        model = JuMP.Model(GLPK.Optimizer)
        @variable(model, x[1:length(WAR_remaining)], Bin) # Whether the player was chosen or not
        (((x, label),) -> set_name(x,label)).(zip(x,String.(dataframe[!,9])))
        @constraint(model, sum(x[1:length(WAR_remaining)]) .== picks) # Round amount
        @constraint(model, sum(salary .* x[1:length(salary)]) .+ total_salary .<= 223000000) # Making sure it doesn't go over the salary cap of 223 Million
         # Making sure there is at least one player taken from each team 
        for col in 31:60
            @constraint(model, sum(dataframe[!, col] .* x) .<= teamlim)
        end  
        # Making sure there is at least two players per position
        for col in 20:27
            @constraint(model, sum(dataframe[!, col] .* x) .>= posmin)
        end     
        # Making sure there are enough pitchers
        for col in 28:29
            @constraint(model, sum(dataframe[!, col] .* x) .>= pitchmin)
        end         
    
        @objective(model, Max, dot(WAR_remaining, x[1:length(WAR_remaining)])   - alpha  *   dot(future_salary, x[1:length(WAR_remaining)]))
        
        optimize!(model) 
        return model, x
    end


    protected_players_df = DataFrame()
    protected_players_dict = Dict{String, DataFrame}() 

    for round in 1:rounds

        picks = picks_tuple[round]
        protect = protect_tuple[round]
        print(round, "\n")

        protected_players_dict = Dict{String, DataFrame}() 

        unprotected_players_df = DataFrame()

        # Each team chooses a list of players that they want to protect 
        function apply_protection(data)
            # Iterate over each team in teams_list
            for team in teams_list
                # Filter protectable players for the current team
                protectable_team_players = filter(row -> row.Selectable == 1 && row.shortName == team, data)
                WAR = protectable_team_players[!,"WAR"]

                # Apply protection model to select top x players by WAR
                model, x = protection_model(WAR, protectable_team_players, protect)
                optimize!(model)
                
                # Check if optimization was successful
                if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
                    protected_players_indices = findall(value.(JuMP.value.(x)) .== 1) 
                    protected_key_bbref_values = protectable_team_players[protected_players_indices, :][:, :key_bbref]
                    # Update the Selectable column in data dataframe where key_bbref matches
                    for key_bbref in protected_key_bbref_values
                        data[data.key_bbref .== key_bbref, :Selectable] .= 0
                    end

                    non_selectable_team_players = filter(row -> row.Selectable == 0 && row.shortName == team, data)
                    protected_players_dict[team] = non_selectable_team_players

                    # Filter out unprotected players for the current team
                    unprotected_players = filter(row -> row.Selectable == 1 && row.shortName == team, data)

                    # Append              
                    append!(unprotected_players_df, unprotected_players)
                    
                else
                    # Team has fewer than x players to protect 
                    println("No more players to protect for $team")

                    protected_players = protectable_team_players  
                    non_selectable_team_players = filter(row -> row.Selectable == 0 && row.shortName == team, data)
                    # Combine protected and non-protectable players into a single DataFrame
                    team_players_df = vcat(protected_players, non_selectable_team_players)
                    protected_players_dict[team] = team_players_df
                end 
            end
        end

        apply_protection(player_data)

        selection_model2, x = selection_model(unprotected_players_df, picks)
        term_status = JuMP.termination_status(selection_model2) 
        selected_players_indices = findall(value.(JuMP.value.(x)) .== 1) 
        selected_players = unprotected_players_df[selected_players_indices, :] 
        total_salary = sum(selected_players[!,"2023"]) 

        selected_key_bbref_values = unprotected_players_df[selected_players_indices, :][:, :key_bbref]
        # Update the Selectable column in data dataframe where key_bbref matches and store all selected IDs 
        for key_bbref in selected_key_bbref_values
            player_data[player_data.key_bbref .== key_bbref, :Selectable] .= 0
            push!(selected_IDs, key_bbref)
        end

        total_salary = sum(selected_players[!,"2023"])
        total_war = sum(selected_players.WAR)
        caphit24 = sum(selected_players[!, "2024Est"])

        selected_players_round = filter(row -> row.key_bbref in selected_key_bbref_values, player_data) 
        clean_selected_players_round = select(selected_players_round, [:Name, :WAR,  :shortName, :SimplePosition])
        selected_players_byround_dict[round] = clean_selected_players_round

    end
    
    selected_players = filter(row -> row.key_bbref in selected_IDs, player_data) 
    total_salary = sum(selected_players[!,"2023"])
    total_war = sum(selected_players.WAR)
    caphit24 = sum(selected_players[!, "2024Est"])
    clean_selected_players = select(selected_players, [:Name, :WAR,  :shortName, :SimplePosition])
    return selected_players, clean_selected_players,   protected_players_dict,  total_war, caphit24, selected_players_byround_dict
end

simulation_rounds (generic function with 1 method)

In [53]:
mod3 = simulation_rounds(rounds_data, (10,3,3), 1, 1, 2, 0, 3, (14,14,7) )  

1


2
No more players to protect for Cleveland


No more players to protect for Oakland
No more players to protect for Detroit
3
No more players to protect for Houston
No more players to protect for Cleveland
No more players to protect for Minnesota
No more players to protect for Arizona


No more players to protect for Pittsburgh
No more players to protect for Kansas City
No more players to protect for Oakland
No more players to protect for Detroit
No more players to protect for Cincinnati


(35×60 DataFrame
 Row │ IDfg   Name                Age    G      AB     WAR      name_last      ⋯
     │ Int64  String31            Int64  Int64  Int64  Float64  String15       ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 14813  Kyle Farmer            30    145    526      1.5  farmer         ⋯
   2 │ 16512  Isiah Kiner-Falefa     26    142    483      1.5  kiner-falefa
   3 │  8202  Josh Harrison          33    119    386      1.4  harrison
   4 │ 19252  Cavan Biggio           26     97    257      1.4  biggio
   5 │ 16556  Ozzie Albies           24     64    247      1.2  albies         ⋯
   6 │ 10950  Adam Duvall            32     86    287      1.0  duvall
   7 │ 17992  David Fletcher         27     61    216      0.7  fletcher
   8 │ 14128  Joey Gallo             27    126    350      0.6  gallo
  ⋮  │   ⋮            ⋮             ⋮      ⋮      ⋮       ⋮          ⋮         ⋱
  29 │ 20367  Brusdar Graterol       22     46      0      0.8

Export results from multi-round draft.

In [54]:
CSV.write("../Results/round1.csv", mod3[6][1])
mod3[6][1]

Row,Name,WAR,shortName,SimplePosition
,String31,Float64,String15,String31?
1,Kyle Farmer,1.5,Minnesota,"3b,ss"
2,Isiah Kiner-Falefa,1.5,NY Yankees,"3b,ss"
3,Josh Harrison,1.4,Philadelphia,"2b,3b"
4,Cavan Biggio,1.4,Toronto,"1b,2b,rf"
5,Ozzie Albies,1.2,Atlanta,2b
6,Adam Duvall,1.0,Boston,"lf,cf,rf"
7,Meibrys Viloria,0.6,Cleveland,c
8,Carlos Carrasco,2.5,NY Mets,Starter
9,Noah Syndergaard,2.2,LA Dodgers,Starter


In [55]:
CSV.write("../Results/round2.csv", mod3[6][2])
mod3[6][2]

Row,Name,WAR,shortName,SimplePosition
,String31,Float64,String15,String31?
1,David Fletcher,0.7,LA Angels,"2b,ss"
2,Joey Gallo,0.6,Minnesota,"lf,rf"
3,Mitch Garver,0.3,Texas,c
4,Christian Arroyo,0.2,Boston,"1b,2b,3b,ss,rf"
5,Mauricio Dubon,0.2,Houston,"2b,ss,lf,cf"
6,Clay Holmes,1.1,NY Yankees,Reliever
7,Adam Ottavino,1.1,NY Mets,Reliever
8,Walker Buehler,1.0,LA Dodgers,Reliever
9,John Brebbia,0.9,San Francisco,Reliever


In [56]:
CSV.write("../Results/round3.csv", mod3[6][3])
mod3[6][3]

Row,Name,WAR,shortName,SimplePosition
,String31,Float64,String15,String31?
1,Tim Locastro,0.0,NY Mets,"lf,cf,rf"
2,Max Stassi,0.0,LA Angels,c
3,Harold Castro,-0.5,Colorado,"1b,2b,3b,ss"
4,Brusdar Graterol,0.8,LA Dodgers,Reliever
5,Matt Strahm,0.3,Philadelphia,Starter
6,Joel Payamps,0.2,Milwaukee,Reliever
7,Hyun-Jin Ryu,0.1,Toronto,Starter
